In [1]:

import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

class SensorNeuron(nn.Module):
    def __init__(self, input_size, dataset_samples, threshold=0.5):
        super(SensorNeuron, self).__init__()
        self.input_size = input_size
        self.dataset_samples = dataset_samples
        self.threshold = threshold  # Limiar para sinal forte
        self.sensory_db = self.initialize_sensory_neurons()  # Banco de dados sensorial
        self.consolidation_counter = [0] * input_size  # Contadores de consolidação
    
    def initialize_sensory_neurons(self):
        sensory_neurons = []
        for _ in range(self.input_size):
            random_sample = random.choice(self.dataset_samples)
            sensory_neurons.append(random.choice(random_sample))  # Neurônios inicializados com dados aleatórios
        return torch.tensor(sensory_neurons).float()
    
    def forward(self, x):
        x = x.float()  # Garantir tipo correto de entrada
        reinforcement_score = 0  # Inicia o score de reforço
        for i in range(self.input_size):
            # Calcula a diferença absoluta
            diff = torch.abs(x[:, i] - self.sensory_db[i])
            
            # Verifica se cada elemento no lote é menor que o threshold
            signal_strong = diff < self.threshold
            
            # Verifica se a maioria dos sinais é forte (True)
            if signal_strong.all():  # Se todos os valores no lote são menores que o limiar
                self.consolidation_counter[i] += 1
                reinforcement_score += 1  # Sinal forte encontrado, reforça a informação
            else:
                self.consolidation_counter[i] -= 1  # Sinal fraco, reduz a consolidação
            
            # Caso o contador de consolidação seja negativo, atualiza o neurônio
            if self.consolidation_counter[i] < 0:
                self.sensory_db = self.update_sensory_neuron(self.sensory_db, self.dataset_samples, i)  # Chama a função externa
        return x, reinforcement_score
    
    def update_sensory_neuron(self, sensory_db, dataset_samples, index):
        random_sample = random.choice(dataset_samples)
        new_value = random.choice(random_sample)
        sensory_db[index] = new_value
        return sensory_db
        
    def reinforce_database(self):
        # Caso o sinal seja encontrado mais de uma vez, a informação é consolidada no banco de dados permanente
        for i in range(self.input_size):
            if self.consolidation_counter[i] > 5:  # Exemplo de limiar de consolidação
                self.sensory_db[i] = self.sensory_db[i]  # Reforço da informação
            elif self.consolidation_counter[i] < -3:
                self.update_sensory_neuron(i)  # Atualiza os neurônios com base no contador de consolidação


    def process_temp_db(self):
        for temp_signal in self.temp_db:
            print(f"temp_signal shape: {temp_signal.shape}")
            print(f"sensory_db shape: {self.sensory_db.shape}")

            if temp_signal.dim() == 1:
                temp_signal = temp_signal.view(1, -1)  # Ajusta a dimensão de temp_signal

            if temp_signal.size(1) == self.sensory_db.size(1):
                diff = torch.abs(temp_signal - self.sensory_db.unsqueeze(0))  # Adiciona uma dimensão ao sensory_db para comparação
                print(f"Diff shape: {diff.shape}")
            else:
                print(f"Dimensões incompatíveis: temp_signal {temp_signal.size()} vs sensory_db {self.sensory_db.size()}")

class AdvancedMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dataset_samples, threshold=0.5):
        super(AdvancedMLP, self).__init__()
        self.sensor_neuron = SensorNeuron(input_size=input_size, dataset_samples=dataset_samples, threshold=threshold)  # Neurônios sensoriais
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x, reinforcement_score = self.sensor_neuron(x)  # Processamento inicial com neurônios sensoriais
        x = F.relu(self.fc1(x))  # Primeira camada de rede neural
        x = self.fc2(x)  # Camada final de saída
        return F.log_softmax(x, dim=1), reinforcement_score



In [2]:
# Carregar o dataset Iris
iris = load_iris()
X = iris.data  # Características (features)
y = iris.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 200

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch [1/200], Loss: 1.2228, Accuracy: 13.33%
Epoch [2/200], Loss: 1.1977, Accuracy: 20.00%
Epoch [3/200], Loss: 1.1701, Accuracy: 20.00%
Epoch [4/200], Loss: 1.1450, Accuracy: 20.00%
Epoch [5/200], Loss: 1.1234, Accuracy: 20.83%
Epoch [6/200], Loss: 1.1028, Accuracy: 20.83%
Epoch [7/200], Loss: 1.0846, Accuracy: 22.50%
Epoch [8/200], Loss: 1.0629, Accuracy: 23.33%
Epoch [9/200], Loss: 1.0421, Accuracy: 24.17%
Epoch [10/200], Loss: 1.0188, Accuracy: 29.17%
Epoch [11/200], Loss: 1.0024, Accuracy: 35.83%
Epoch [12/200], Loss: 0.9849, Accuracy: 44.17%
Epoch [13/200], Loss: 0.9648, Accuracy: 53.33%
Epoch [14/200], Loss: 0.9466, Accuracy: 57.50%
Epoch [15/200], Loss: 0.9264, Accuracy: 60.83%
Epoch [16/200], Loss: 0.9060, Accuracy: 62.50%
Epoch [17/200], Loss: 0.8900, Accuracy: 62.50%
Epoch [18/200], Loss: 0.8764, Accuracy: 63.33%
Epoch [19/200], Loss: 0.8600, Accuracy: 63.33%
Epoch [20/200], Loss: 0.8431, Accuracy: 64.17%
Epoch [21/200], Loss: 0.8226, Accuracy: 65.00%
Epoch [22/200], Loss: 

In [3]:
# Carregar o dataset Iris
digits = load_digits()
X = digits.data  # Características (features)
y = digits.target  # Rótulos (labels)

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores do PyTorch
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Criar DataLoader para treino
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Criar DataLoader para teste
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Definir o modelo
input_size = X_train.shape[1]  # 4 características de entrada
hidden_size = 16
output_size = len(set(y))  # 3 classes (Setosa, Versicolor, Virginica)
dataset_samples = [list(range(10))]  # Exemplo simples, substitua conforme necessário
threshold = 0.5

model = AdvancedMLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, dataset_samples=dataset_samples, threshold=threshold)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()  # Para classificação
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_preds = 0
    total_samples = 0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        
        # Forward pass
        outputs, reinforcement_score = model(inputs)
        
        # Calcular a perda
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calcular acurácia
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct_preds / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Avaliar o modelo nos dados de teste
model.eval()
correct_preds = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs, _ = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_samples += labels.size(0)

test_accuracy = 100 * correct_preds / total_samples
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch [1/100], Loss: 2.1572, Accuracy: 24.63%
Epoch [2/100], Loss: 1.7826, Accuracy: 52.12%
Epoch [3/100], Loss: 1.3857, Accuracy: 68.48%
Epoch [4/100], Loss: 1.0367, Accuracy: 77.31%
Epoch [5/100], Loss: 0.7769, Accuracy: 83.92%
Epoch [6/100], Loss: 0.6006, Accuracy: 86.99%
Epoch [7/100], Loss: 0.4813, Accuracy: 89.35%
Epoch [8/100], Loss: 0.3978, Accuracy: 91.86%
Epoch [9/100], Loss: 0.3332, Accuracy: 93.39%
Epoch [10/100], Loss: 0.2840, Accuracy: 94.85%
Epoch [11/100], Loss: 0.2442, Accuracy: 95.82%
Epoch [12/100], Loss: 0.2139, Accuracy: 96.10%
Epoch [13/100], Loss: 0.1901, Accuracy: 96.38%
Epoch [14/100], Loss: 0.1705, Accuracy: 96.87%
Epoch [15/100], Loss: 0.1542, Accuracy: 97.29%
Epoch [16/100], Loss: 0.1407, Accuracy: 97.49%
Epoch [17/100], Loss: 0.1293, Accuracy: 97.77%
Epoch [18/100], Loss: 0.1189, Accuracy: 97.98%
Epoch [19/100], Loss: 0.1097, Accuracy: 98.19%
Epoch [20/100], Loss: 0.1020, Accuracy: 98.19%
Epoch [21/100], Loss: 0.0952, Accuracy: 98.40%
Epoch [22/100], Loss: 